In [1]:
from scrapy.spider import BaseSpider

In [14]:
class MySpider(BaseSpider):
    name = "uni-kl.de"
    allowed_domains = ["uni-kl.de"]
    start_urls = [
        "http://www.uni-kl.de/",
    ]

    def parse(self, response):
        filename = response.url.split("/")[-2]
        open(filename, 'wb').write(response.body)

In [17]:
# see: http://stackoverflow.com/questions/7993680/running-scrapy-tasks-in-python

from scrapy import project, signals
from scrapy.conf import settings
from scrapy.crawler import CrawlerProcess
from scrapy.xlib.pydispatch import dispatcher
from multiprocessing.queues import Queue
from multiprocessing import Process

class CrawlerWorker(Process):
    def __init__(self, spider, results):
        Process.__init__(self)
        self.results = results

        self.crawler = CrawlerProcess(settings)
        if not hasattr(project, 'crawler'):
            self.crawler.install()
        self.crawler.configure()

        self.items = []
        self.spider = spider
        dispatcher.connect(self._item_passed, signals.item_passed)

    def _item_passed(self, item):
        self.items.append(item)

    def run(self):
        self.crawler.crawl(self.spider)
        self.crawler.start()
        self.crawler.stop()
        self.results.put(self.items)


In [18]:
# The part below can be called as often as you want
results = Queue()
crawler = CrawlerWorker(MySpider(), results)
crawler.start()
for item in results.get():
    print item